# Тестовое задание.
## Определить абонентов (msisdn), которые являются одной и той же Персоной (человеком).

In [1]:
import pandas as pd
import os
from os import walk
import datetime
pd.set_option('display.max_columns', 500)

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime, date

import csv

import folium
from folium import plugins
from folium.plugins import HeatMap
import branca
import branca.colormap as cm

Загружаем данные и преобразуем в удобный формат некоторые поля

In [2]:
current_directory = r'C:\Users\ekaterina.adischeva\Documents\Scripts\BD_task\Техническое задание_аналитик больших данных\Данные'
current_directory = current_directory.replace('\\','/')
os.chdir(current_directory)

In [3]:
df_data = pd.read_csv('02_Data.csv', sep = ';', dtype = {'imei': str, 'lac' : 'str', 'cid' : 'str'})
facts = pd.read_excel('01_Факты.xlsx', header = None)
tac_dict = pd.read_csv('03_устройства.csv', quoting=csv.QUOTE_NONE)
event_type = pd.read_excel('04_event_type.xlsx')

In [4]:
tac_dict.columns = tac_dict.columns.str.replace('\"','')
for column_name in tac_dict.columns:
    tac_dict[column_name] = tac_dict[column_name].str.replace('\"','')

Преобразуем в datetime время регистрации события на базовой станции (количество 1/1000 секунды, прошедших с 01.01.1970 до времени регистрации на БС.)

Выделяем tac из поля imei 

In [5]:
df_data['datetime'] = pd.to_timedelta(df_data['tstamp']/1000, unit='S') + pd.Timestamp('1970-1-1')
df_data['datetime'] = pd.to_datetime(df_data['datetime'])

df_data['tac'] = df_data['imei'].str.slice(0,8)

Подгружаем значения из справочников в данные о перемещении абонентов 

In [7]:
df_data = df_data.merge(event_type, left_on = ['event_type'], right_on = 'Номер', how = 'left')
df_data = df_data.merge(tac_dict, on = ['tac'], how = 'left')

In [ ]:
'''
def define_rectangle(long = 0., lat = 0., r = 0., angle_1 = 0., angle_2 = 0.):
    
    long_ = [0, r*np.cos(angle_1), r*np.cos(angle_2)]
    lat_ = [0, r*np.sin(angle_1), r*np.sin(angle_2)]
    
    return long+np.min(long_)/62/1000, long+np.max(long_)/62/1000, lat+np.min(lat_)/111/1000, lat+np.max(lat_)/111/1000

df_data[['long_min','long_max', 'lat_min','lat_max']] = df_data.apply(lambda x: pd.Series(define_rectangle(x['long'],
                                                                                                x['lat'],
                                                                                                x['max_dist'],
                                                                                                x['start_angle'],
                                                                                                x['end_angle'])),axis = 1)
'''

In [ ]:
def haversine_array(lat1, lng1, lat2, lng2):
    lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2))
    AVG_EARTH_RADIUS = 6371  # in km
    lat = lat2 - lat1
    lng = lng2 - lng1
    d = np.sin(lat * 0.5) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(lng * 0.5) ** 2
    h = 2 * AVG_EARTH_RADIUS * np.arcsin(np.sqrt(d))
    return h
lat_length = haversine_array(55.2522200, 37.6155600, 56.2522200, 37.6155600)
long_length = haversine_array(55.7522200, 37.1155600, 55.7522200, 38.1155600)

In [ ]:
df_data['long_start'] = df_data['long'] + df_data['max_dist'] * np.sin(df_data['start_angle'])/long_length/1000
df_data['long_end'] = df_data['long'] + df_data['max_dist'] * np.sin(df_data['end_angle'])/long_length/1000
df_data['long_min'] = df_data[['long_start', 'long_end', 'long']].min(axis = 1)
df_data['long_max'] = df_data[['long_start', 'long_end', 'long']].max(axis = 1)

In [ ]:
df_data['lat_start'] = df_data['lat'] + df_data['max_dist'] * np.cos(df_data['start_angle'])/lat_d/1000
df_data['lat_end'] = df_data['lat'] + df_data['max_dist'] * np.cos(df_data['end_angle'])/lat_d/1000
df_data['lat_min'] = df_data[['lat_start', 'lat_end', 'lat']].min(axis = 1)
df_data['lat_max'] = df_data[['lat_start', 'lat_end', 'lat']].max(axis = 1)

In [ ]:
def show_chart(fig, data_a, data_b ):
    """
    The function draws 2 chatrs: msisdn lattitude on time and msisdn longitude on time

"""

    data_a_copy = data_a
    data_b_copy = data_b

    plt.subplot(1, 2, 1)
    feature = 'long'
    plt.scatter(data_a_copy['datetime'], data_a_copy[feature], alpha = .5, c = 'blue')
    plt.plot(data_a_copy['datetime'], data_a_copy[feature+'_max'],'-o',alpha = .3, color='blue')
    plt.plot(data_a_copy['datetime'], data_a_copy[feature+'_min'],'-o',alpha = .3, color='blue')
    plt.scatter(data_b_copy['datetime'], data_b_copy[feature], alpha = .5, c = 'red')
    plt.plot(data_b_copy['datetime'], data_b_copy[feature+'_max'],'-o',alpha = .3, color='red')
    plt.plot(data_b_copy['datetime'], data_b_copy[feature+'_min'],'-o',alpha = .3, color='red')
    plt.xlim(pd.concat([data_a,data_b])['datetime'].min(), pd.concat([data_a,data_b])['datetime'].max())
    plt.xticks(rotation=70)

    plt.subplot(1, 2, 2)
    feature = 'lat'
    plt.scatter(data_a_copy['datetime'], data_a_copy[feature], alpha = .5, c = 'blue')
    plt.plot(data_a_copy['datetime'], data_a_copy[feature+'_max'],'-o',alpha = .3, color='blue')
    plt.plot(data_a_copy['datetime'], data_a_copy[feature+'_min'],'-o',alpha = .3, color='blue')
    plt.scatter(data_b_copy['datetime'], data_b_copy[feature], alpha = .5, c = 'red')
    plt.plot(data_b_copy['datetime'], data_b_copy[feature+'_max'],'-o',alpha = .3, color='red')
    plt.plot(data_b_copy['datetime'], data_b_copy[feature+'_min'],'-o',alpha = .3, color='red')
    plt.xlim(pd.concat([data_a,data_b])['datetime'].min(), pd.concat([data_a,data_b])['datetime'].max())
    plt.xticks(rotation=70)
    
    plt.show()
    return fig

In [ ]:
def show_circles_on_map(m, data, latitude_column, longitude_column, color):
    """
    The function draws map with circles on it.
    The center of the map is the mean of coordinates passed in data.
    
    data: DataFrame that contains columns latitude_column and longitude_column
    latitude_column: string, the name of column for latitude coordinates
    longitude_column: string, the name of column for longitude coordinates
    color: string, the color of circles to be drawn
    """

    location = (data[latitude_column].mean(), data[longitude_column].mean())

    for _, row in data.iterrows():
        folium.Circle(
            radius=100,
            location=(row[latitude_column], row[longitude_column]),
            color=color,
            fill_color=color,
            fill=True,
            fill_opacity = .5
        ).add_to(m)

    return m

In [ ]:
# for 2 persons we don't have second msisdn in our dataset
display(facts[~facts[1].isin(df_data['msisdn'])], 
        facts[~facts[0].isin(df_data['msisdn'])])

In [ ]:
plt.hist(df_data['datetime'].values, bins=100)
plt.xticks(rotation=70)
plt.show()

In [ ]:
m = folium.Map(location=(55.7522200, 37.6155600))
show_circles_on_map(m, df_data.sample(500), "lat", "long", "blue")

In [ ]:
df_data.loc[:, 'date'] = df_data['datetime'].dt.date

In [ ]:
df_data['date'].value_counts()

In [ ]:
df_data = df_data[(df_data['datetime']  < datetime.strptime('2013-05-26', '%Y-%m-%d'))
         #         & (df_data['datetime']  >= datetime.strptime('2013-05-23', '%Y-%m-%d'))
                 ]

df_data['date'].value_counts()

In [ ]:
df_data['angel'] = np.where(df_data['end_angle'] < df_data['start_angle'], df_data['end_angle'] - df_data['start_angle'] + 360,
                        df_data['end_angle'] - df_data['start_angle'])

In [ ]:
df_data['angel'].value_counts()

In [ ]:
plt.hist(df_data['max_dist'].values, bins=20)
plt.xticks(rotation=70)
plt.show()

In [ ]:
'''
bs_agg = df_data.groupby(['lat', 'long'])['msisdn'].count().reset_index()
bs_agg = bs_agg.sample(770)


bs_agg_min = bs_agg['msisdn'].min()
bs_agg_max = bs_agg['msisdn'].max()
print(bs_agg_min, bs_agg_max)

m = folium.Map(location=(55.7522200, 37.6155600)) 
colormap = cm.LinearColormap(colors=['lightblue','red'],
                             index=[bs_agg_min,bs_agg_max],
                             vmin=bs_agg_min,vmax=bs_agg_max)


lat = list(bs_agg.lat)
lon = list(bs_agg.long)
pow = list(bs_agg.msisdn)

for loc, p in zip(zip(lat, lon), pow):
    folium.Circle(
        location=loc,
        radius=1000,
        fill=True,
        color='red',
        fill_opacity=0.1
    ).add_to(m)

m.add_child(colormap)

display(m)
'''

In [ ]:
df_data['hour'] = df_data.datetime.dt.hour
df_data['day_of_month'] = df_data.datetime.dt.day
df_data['minute_of_hour'] = df_data.datetime.dt.minute

In [ ]:
sns.relplot(data=df_data.groupby(['hour', 'day_of_month'])['lac'].count().reset_index(),
            y="lac", x="hour", hue = 'day_of_month')

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.datasets import make_blobs

In [ ]:
X = df_data['datetime'].values.tolist()
X = np.reshape(X, (-1, 2))

In [ ]:
BINS_QNT = 23
strategies = ['quantile']
for strategy in strategies:
        enc = KBinsDiscretizer(n_bins=BINS_QNT, encode='ordinal', strategy=strategy)
        enc.fit(X)
        Xt = enc.transform(X)
        df_data['datetime_bin'] = Xt.ravel()

In [ ]:
df_data[df_data['datetime_bin'] == 89.].groupby(['day_of_month', 'hour'])['lac'].count().reset_index()

In [ ]:
sns.relplot(data=df_data.groupby(['datetime_bin', 'day_of_month'])['lac'].count().reset_index(),
            y="lac", x='datetime_bin', hue = 'day_of_month')

In [ ]:
df_data_agg = df_data.groupby(['msisdn',  'datetime_bin'])['long_min', 'long_max', 'lat_min', 'lat_max', 'long', 'lat'].mean().reset_index()

In [ ]:
df_data_agg['datetime'] = df_data_agg['datetime_bin'] 

In [ ]:
a = df_data_agg['msisdn'].drop_duplicates().reset_index().sample(n = 1)['msisdn'].iloc[0]
b = df_data_agg['msisdn'].drop_duplicates().reset_index().sample(n = 1)['msisdn'].iloc[0]
a, b = 158527183389, 158503655953
a, b = 158592156963, 158534344291
print('Random', a, b)
fig = plt.figure(figsize=(15, 4))
show_chart(fig, df_data[df_data['msisdn'] == a], df_data[df_data['msisdn'] == b])
plt.show()
fig = plt.figure(figsize=(15, 4))
show_chart(fig, df_data_agg[df_data_agg['msisdn'] == a], df_data_agg[df_data_agg['msisdn'] == b])
plt.show()

m = folium.Map(location=(55.7522200, 37.6155600)) 
show_circles_on_map(m, df_data_agg[df_data_agg['msisdn'] == a ], "lat", "long", "blue")
show_circles_on_map(m, df_data_agg[df_data_agg['msisdn'] == b ], "lat", "long", "red")


In [ ]:
a, b = facts.sample(n = 1).iloc[0]
a, b = predict_error.sample(1).iloc[0][['msisdn_x', 'msisdn_y']]

print('One person', a, b)
fig = plt.figure(figsize=(15, 4))
show_chart(fig, df_data[df_data['msisdn'] == a], df_data[df_data['msisdn'] == b])
plt.show()
fig = plt.figure(figsize=(15, 4))
show_chart(fig, df_data_agg[df_data_agg['msisdn'] == a], df_data_agg[df_data_agg['msisdn'] == b])
plt.show()

m = folium.Map(location=(55.7522200, 37.6155600)) 
show_circles_on_map(m, df_data_agg[df_data_agg['msisdn'] == a ], "lat", "long", "blue")
show_circles_on_map(m, df_data_agg[df_data_agg['msisdn'] == b ], "lat", "long", "red")


In [ ]:
df_data[df_data['msisdn'].isin([a,b])][['msisdn', 'imei', 'tac']].drop_duplicates()

In [ ]:
df_data_agg['long'] = (df_data_agg['long_max'] + df_data_agg['long_min'])/2
df_data_agg['lat'] = (df_data_agg['lat_max'] + df_data_agg['lat_min'])/2

In [ ]:
df_data_agg = df_data_agg.sort_values(['msisdn', 'datetime_bin'])
df_data_agg['K'] = df_data_agg.groupby(['msisdn'])['msisdn'].cumcount()
df_data_agg['K1'] = df_data_agg['K'] - 1

In [ ]:
df_data_agg_full = df_data_agg.merge(df_data_agg[['msisdn', 'long', 'lat', 'datetime', 'K1']],
                                    left_on = ['msisdn', 'K'],
                                    right_on = ['msisdn', 'K1'], how = 'left')

In [ ]:
df_data_agg_full.head()

In [ ]:
df_data_msisdn = df_data_agg['msisdn'].drop_duplicates().reset_index()
df_data_datetime = df_data_agg['datetime_bin'].drop_duplicates().reset_index()
df_data_msisdn['K'] = 1
df_data_datetime['K'] = 1
df_data_agg_linear = df_data_msisdn.merge(df_data_datetime, on = ['K']).reset_index()

In [ ]:
df_data_agg_linear = df_data_agg_linear[['msisdn', 'datetime_bin']].set_index(['msisdn', 'datetime_bin'])

In [ ]:
df_data_agg_linear = pd.concat([df_data_agg_linear, df_data_agg_full.set_index(['msisdn', 'datetime_bin'])], axis = 1)
df_data_agg_linear.reset_index(inplace = True)

In [ ]:
df_data_agg_linear.head(10)

In [ ]:
df_data_agg_linear['xx'] = 0
df_data_agg_linear['xx'] = 1- (df_data_agg_linear['datetime_x'].isna())*1

In [ ]:
df_data_agg_linear['datetime_xx'] =  df_data_agg_linear.groupby(['msisdn'])['xx'].cumsum()
df_data_agg_linear['datetime_xx'] = df_data_agg_linear['datetime_xx'] -1
df_data_agg_linear_ = df_data_agg_linear[['msisdn', 'datetime_bin', 'datetime_xx']].merge(df_data_agg_full, left_on = ['msisdn', 'datetime_xx'], right_on = ['msisdn', 'K'], how = 'left')

In [ ]:
df_data_agg_linear_['long'] = df_data_agg_linear_['long_x'] + \
    (df_data_agg_linear_['datetime_bin_x'] - df_data_agg_linear_['datetime_x'])/(df_data_agg_linear_['datetime_y'] - df_data_agg_linear_['datetime_x'])*((df_data_agg_linear_['long_y'] - df_data_agg_linear_['long_x']))
df_data_agg_linear_['lat'] = df_data_agg_linear_['lat_x'] + \
    (df_data_agg_linear_['datetime_bin_x'] - df_data_agg_linear_['datetime_x'])/(df_data_agg_linear_['datetime_y'] - df_data_agg_linear_['datetime_x'])*((df_data_agg_linear_['lat_y'] - df_data_agg_linear_['lat_x']))

In [ ]:
df_data_agg_linear_['long'] = np.where(df_data_agg_linear_['datetime_bin_x'] == df_data_agg_linear_['datetime_x'], 
                                        df_data_agg_linear_['long_x'], df_data_agg_linear_['long'] )
df_data_agg_linear_['lat'] = np.where(df_data_agg_linear_['datetime_bin_x'] == df_data_agg_linear_['datetime_x'], 
                                        df_data_agg_linear_['lat_x'], df_data_agg_linear_['lat'] )

In [ ]:
df_data_agg_linear_.rename(columns = {'datetime_bin_x':'datetime'}, inplace = True)
df_data_agg_linear_ = df_data_agg_linear_[['msisdn', 'datetime', 'lat', 'long']]

In [ ]:
df_data_agg_lat = df_data_agg_linear_.pivot(index = 'msisdn', columns = 'datetime', values = 'lat')
df_data_agg_lat.columns = df_data_agg_lat.columns.astype('str') + '_lat'
df_data_agg_long = df_data_agg_linear_.pivot(index = 'msisdn', columns = 'datetime', values = 'long')
df_data_agg_long.columns = df_data_agg_long.columns.astype('str') + '_long'

In [ ]:
df_data_agg_coord = pd.concat([df_data_agg_lat, df_data_agg_long], axis = 1).reset_index()
df_data_agg_coord.head()

In [ ]:
df_data_agg_coord['N'] = 1

In [ ]:
train_msisdn_2sim = pd.concat([facts[[0, 1]].rename(columns = {0:'msisdn_x', 1:'msisdn_y'}),
                          facts[[1, 0]].rename(columns = {1:'msisdn_x', 0:'msisdn_y'})], axis = 0) 
print(train_msisdn_2sim.shape, train_msisdn_2sim['msisdn_x'].nunique())
train_msisdn_2sim['y'] = 1

In [ ]:
msisdns_pairs = df_data_msisdn.merge(df_data_msisdn, on = ['K']).drop(columns = ['index_x', 'index_y', 'K'])
msisdns_pairs = msisdns_pairs[msisdns_pairs['msisdn_x'] != msisdns_pairs['msisdn_y']]
msisdns_pairs = msisdns_pairs.merge(train_msisdn_2sim, on = ['msisdn_x', 'msisdn_y'], how = 'left')
msisdns_pairs['y'] = np.where(msisdns_pairs['msisdn_x'] == msisdns_pairs['msisdn_y'], 1, msisdns_pairs['y'])
msisdns_pairs['y'] = np.where(msisdns_pairs['msisdn_x'].isin(train_msisdn_2sim['msisdn_x']) &
                              ~(msisdns_pairs['msisdn_y'].isin(train_msisdn_2sim['msisdn_x'])), 0, msisdns_pairs['y'])
msisdns_pairs['y'] = np.where(msisdns_pairs['msisdn_y'].isin(train_msisdn_2sim['msisdn_x']) &
                              ~(msisdns_pairs['msisdn_x'].isin(train_msisdn_2sim['msisdn_x'])), 0, msisdns_pairs['y'])

In [ ]:
msisdns_pairs['y'].value_counts(), msisdns_pairs.shape

In [ ]:
train_msisdn = msisdns_pairs[~msisdns_pairs['y'].isna()]
train_msisdn.shape

In [ ]:
df_data_agg_coord.drop(columns=['N'], inplace = True)

In [ ]:
train = train_msisdn.merge(df_data_agg_coord.rename(columns = {'msisdn':'msisdn_x'}), on = ['msisdn_x']).merge(df_data_agg_coord.rename(columns = {'msisdn':'msisdn_y'}), on = ['msisdn_y'])

In [ ]:
train.shape, train.columns

In [ ]:
train.columns = train.columns.str.replace('.0_', '_')

In [ ]:
for i in df_data['datetime_bin'].unique():
    str_i = str(int(i))
    lat_diff = (train[str_i+"_lat_x"].values - train[str_i+"_lat_y"].values)/lat_length
    long_diff = (train[str_i+"_long_x"].values - train[str_i+"_long_y"].values)/long_length
    length = np.sqrt(lat_diff**2 + long_diff**2)
    train[str_i+"_dist"] = length

In [ ]:
train['y'].value_counts()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split


X = train.drop(columns = ['msisdn_x', 'msisdn_y', 'y']).fillna(0)
y = train['y']

X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42)

clf = RandomForestClassifier(random_state=0)
clf.fit(X_train, y_train)

In [ ]:
'''
param_grid = dict( )
pipeline=Pipeline([ ('clf',clf) ])
strarifiedCV = StratifiedShuffleSplit(n_splits=10,test_size=0.2, random_state=0)
grid_search = GridSearchCV(pipeline,param_grid=param_grid,verbose=3,scoring='accuracy',
'''

In [ ]:
y_test_predict = clf.predict(X_test)
y_train_predict = clf.predict(X_train)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score

print('Train accuracy:', accuracy_score(y_train, y_train_predict), 'roc_auc_score:', roc_auc_score(y_train, y_train_predict),
      'average_precision_score:', average_precision_score(y_train, y_train_predict))
print('Test accuracy:', accuracy_score(y_test, y_test_predict), "roc_auc_score:", roc_auc_score(y_test, y_test_predict),
      'average_precision_score:', average_precision_score(y_test, y_test_predict))

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import roc_auc_score

disp = plot_roc_curve(clf, X_test, y_test)
roc_auc = roc_auc_score(y_test, y_test_predict)
print(roc_auc)

In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve

disp = plot_precision_recall_curve(clf, X_test, y_test)
from sklearn.metrics import average_precision_score
average_precision = average_precision_score(y_test, y_test_predict)
print(average_precision)

In [ ]:
y_predict = clf.predict(X)

In [ ]:
y_test_predict[502225]

In [ ]:
diff = (y_test - y_test_predict).reset_index()

In [ ]:
predict_error = train.iloc[diff[diff['y'] != 0]['index']]

# phone models

In [ ]:
df_data_devices = df_data.groupby(['tac','vendor', 'platform', 'type'])['msisdn'].count().reset_index().sort_values('msisdn', ascending = False)

In [ ]:
df_data_devices_ms = df_data[df_data['msisdn'].isin(train_msisdn['msisdn_x'])].groupby(['tac','vendor', 'platform', 'type'])['msisdn'].count().reset_index().sort_values('msisdn', ascending = False)

In [ ]:
A = df_data_devices['msisdn'].sum()
df_data_devices['msisdn'] = df_data_devices['msisdn']/A
df_data_devices.head(20)

In [ ]:
A = df_data_devices_ms['msisdn'].sum()
df_data_devices_ms['msisdn'] = df_data_devices_ms['msisdn']/A
df_data_devices_ms.head(20)

In [ ]:
df_data.groupby(['tac','vendor', 'platform', 'type'])['msisdn'].count().reset_index().sort_values('msisdn', ascending = False)